In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
cd '/content/drive/My Drive/PLANT DISEASE RECOGNITION'

In [ ]:
!mkdir config datasets models

In [ ]:
cd config

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/PLANT DISEASE RECOGNITION/config"

In [ ]:
!ls

In [ ]:
cd '/content/drive/My Drive/PLANT DISEASE RECOGNITION/datasets'

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
!ls

In [ ]:
# check the dataset folder size
!du -hs 'New Plant Diseases Dataset(Augmented)'

In [ ]:
ls

In [ ]:
# !du -hs 'new plant diseases dataset(augmented)'

In [ ]:
# !du -hs 'test'

In [ ]:
# !unzip -nqq "/content/drive/My Drive/PLANT DISEASE RECOGNITION/datasets/new-plant-diseases-dataset.zip" -d "/content/drive/My Drive/PLANT DISEASE RECOGNITION/datasets"

In [ ]:
#Unzipping the zip files to extract the dataset folder and deleting the zip files
# !unzip \*.zip  && rm *.zip

In [ ]:
# !rm -rf New\ Plant\ Diseases\ Dataset\(Augmented\)

In [ ]:
!pwd

In [ ]:
# for (root,dirs,files) in os.walk('.', topdown = True):
#   print(root, dirs)

In [ ]:
# for (root,dirs,files) in os.walk('.', topdown = True):
#   print(root, dirs)

In [ ]:
import os
base_dir = './New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'
os.listdir(base_dir)

In [ ]:
len(os.listdir(os.path.join(base_dir, 'train')))

## Importing the required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import PIL
import tensorflow as tf
from tensorflow.python import keras
import warnings 
import argparse
warnings.filterwarnings('ignore')
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
fig,axes = plt.subplots(1,5, figsize=(18,18))
images = os.listdir(os.path.join(base_dir, 'train/Apple___Black_rot'))
for _ in range(5):
  ax = axes[_]
  image_path = base_dir+'/train/Apple___Black_rot/'+images[_]
  img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
  ax.imshow(img)
  ax.axis('off')
plt.show()

## Data Augmentation transformations for the train dataset

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

# Data augmentation for validation dataset
validation_datagen = ImageDataGenerator(rescale = 1./255)

## Loading the images from their directories

In [ ]:

BATCH_SIZE = 32

train_set_from_dir = train_datagen.flow_from_directory(os.path.join(base_dir, 'train'),
                                                       target_size = (224,224),
                                                       batch_size = BATCH_SIZE,
                                                       class_mode = 'categorical'
                                                       )

validation_set_from_dir = validation_datagen.flow_from_directory(os.path.join(base_dir, 'valid'),
                                                                 target_size = (224,224),
                                                                 batch_size = BATCH_SIZE,
                                                                 class_mode = 'categorical'
)

In [ ]:
TRAIN_SIZE = train_set_from_dir.n
VALID_SIZE = validation_set_from_dir.n

In [ ]:
classes_dict = train_set_from_dir.class_indices

In [ ]:
img = train_set_from_dir.filepaths[np.random.random_integers(low=0, high=train_set_from_dir.samples)]
img = cv2.imread(img)
plt.imshow(img)

In [ ]:
import json
# with open('/content/drive/My Drive/PLANT DISEASE RECOGNITION/class_indices.json','w') as f:
  # json.dump(classes_dict, f)

## Building a simple CNN Architecture

REFERENCES :- https://heartbeat.fritz.ai/regularization-techniques-for-image-processing-using-tensorflow-56c5b365bc17

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import L1L2
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
model = Sequential()
# Convolutional 1st layer
model.add(Conv2D(32, kernel_size = (7,7), activation='relu', input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional 2nd layer
model.add(Conv2D(64, kernel_size = (5,5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

# Convolutional 3rd layer
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.2))

# Convolutional 4th layer
model.add(Conv2D(128, kernel_size=(3,3), activation= 'relu', padding='same'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

# Fully connected 1st layer
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

# Fully connected 2nd layer
model.add(Dense(1600, activation='relu'))
model.add(Dropout(0.2))

# Fully connected 3rd layer
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))

# Fully connected semi-final layer
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))

# Fully connected final layer (for 38 distinct categories hence 38 neurons)
model.add(Dense(38, activation = 'softmax'))

In [ ]:
INIT_LR = 3e-4
model.compile(optimizer = Adam(learning_rate = INIT_LR),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
)

In [ ]:
model.summary()

In [ ]:
## Setting up callbacks for our model

callbacks = [
           ModelCheckpoint('/content/drive/My Drive/PLANT DISEASE RECOGNITION/checkpoints/cnnmodel_plantdiseases.h5', save_best_only=True, monitor='val_accuracy'),
           EarlyStopping(monitor='val_loss', patience=10, verbose=1),
           ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1) 
]

In [ ]:
N_EPOCHS = 16

history = model.fit(train_set_from_dir,
          validation_data = validation_set_from_dir,
          epochs = N_EPOCHS,
          # # Use 128 random batches for training set 
          steps_per_epoch = 128, # 150 x 32 =  random samples
          # # Use 64 random batches for training validation set
          validation_steps = 64, # 100 x 32 = 3200 random samples 
          callbacks = callbacks
          )

### Transfer learning with MobileNet architecture

In [ ]:
# Importing the required libraries for modelling 
from keras import Input, Model
from keras.applications import MobileNet
from keras.layers.core import Flatten, Dense,Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
base_model = MobileNet(
    #Load weights into the pre-trained MobileNet model
    weights="imagenet",
    input_shape=(224, 224, 3),
    #Exclude the ImageNet classifier at the top of the model
    include_top=False
)  

In [ ]:
base_model.summary()

In [ ]:
head_model = base_model.output
head_model = GlobalAveragePooling2D()(head_model)
# Regularization by applying DropOut
head_model = Dropout(0.2)(head_model)
outputs = Dense(38, activation="softmax")(head_model)
mobilenet_model = Model(base_model.input, outputs, name='pretrained_mobilenet' )

for layer in mobilenet_model.layers:
    layer.trainable = False

# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
# Compiling the model with the optimizer and loss function 

mobilenet_model.compile(optimizer = Adam(),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
)

In [ ]:
mobilenet_model.summary()
for idx, layer in enumerate(mobilenet_model.layers):
    print(idx, layer.name, layer.trainable)

In [ ]:
## Setting up callbacks for our model

callbacks = [
           ModelCheckpoint('content/drive/My Drive/PLANT DISEASE RECOGNITION/checkpoints/mobilenet_plantdiseases.h5', save_best_only=True, monitor='val_acc'),
           EarlyStopping(monitor='val_loss', patience=2, verbose=1),
           ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1) 
]

In [ ]:
N_EPOCHS = 10

history = mobilenet_model.fit(train_set_from_dir,
          validation_data = validation_set_from_dir,
          epochs = N_EPOCHS,
          # Use 128 random batches for training set 
          steps_per_epoch = 128, # 128 x 32 = 2**12 random samples
          # Use 64 random batches for training validation set
          validation_steps = 100, # 100 x 32 = 3200 random samples 
          callbacks = callbacks
          )

In [ ]:
n = 6
plt.figure(figsize = (8,5))
plt.plot(np.arange(1,n+1), history.history['loss'], label = 'train_loss')
plt.plot(np.arange(1,n+1), history.history['val_loss'], label = 'val_loss')
plt.plot(np.arange(1,n+1), history.history['accuracy'], label = 'train_accuracy')
plt.plot(np.arange(1,n+1), history.history['val_accuracy'], label = 'val_accuracy')

plt.grid(True)
plt.legend(loc = "best")
plt.savefig('/content/drive/My Drive/PLANT DISEASE RECOGNITION/performance.jpg')
plt.show()

In [ ]:
# Evaluate the model on the Validation dataset
results = mobilenet_model.evaluate(validation_set_from_dir)

In [ ]:
print("Validation Loss :-", results[0])
print("="*30)
print("Validation Accuracy :-", results[1])

In [ ]:
cd '/content/drive/My Drive/PLANT DISEASE RECOGNITION'

In [ ]:
mkdir models

In [ ]:
# Save model in HDF5 format
mobilenet_model.save('models/mobilenet_model.h5')

In [ ]:
# Testing on a random image from the test images directory
from PIL import Image
np.random.seed(200)
idx = np.random.randint(30)
test_images_dir = os.path.join('datasets/test', 'test')
test1 = Image.open(os.path.join(test_images_dir, os.listdir(test_images_dir)[idx]))

In [ ]:
plt.imshow(test1)
plt.title(os.listdir(test_images_dir)[idx])

In [ ]:
test1 = test1.resize((224,224))
test1_scaled = np.expand_dims(np.asarray(test1), axis = 0) / 255
predictions = mobilenet_model.predict(test1_scaled)
print(predictions)

In [ ]:
classes_dict = train_set_from_dir.class_indices
classes_dict = { v:k for (k,v) in classes_dict.items() }
classes_dict[np.argmax(predictions)]

In [ ]:
from tensorflow.python.keras.models import load_model
mobilenet_model = load_model('models/mobilenet_model.h5')

In [ ]:
mobilenet_model.summary()

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model)
tflite_model = converter.convert()
open("models/converted_model.tflite", "wb").write(tflite_model)

#### Accurate !

# Converting model to Tensorflow js

In [ ]:
!pip install tensorflowjs

In [ ]:
!pwd

In [ ]:
!mkdir tensorflowjs-model

In [ ]:
!tensorflowjs_converter --input_format keras models/mobilenet_model.h5 tensorflowjs-model/

In [ ]:
!zip -r tensorflowjs-model.zip tensorflowjs-model

## And we are done !